# 침수 API용 데이터 구축

In [2]:
import pandas as pd 
import glob
from tqdm import tqdm
import dask.dataframe as dd

### (1) kisti가 제공하는 침수된 건물 정보

In [8]:
# KISTI 침수된 건물 정보

path = "data/rawdata/flooding-kisti-data/*.csv"

# Merge all files
flood_df = pd.concat([pd.read_csv(f, encoding="utf-8", keep_default_na=False) for f in glob.glob(path)])
flood_df = flood_df.drop(["OBJECTID_1", "OBJECTID", "full_pk", "pnu_1", "Shape_Length", "Shape_Area"], axis=1)

flood_df.columns

Index(['PNU', 'BD_MGT_SN', '시군구코드', '법정동코드', '번', '지', '관리건축물대장PK', '건물명',
       '새주소도로코드', '새주소법정동코드', '새주소지상지하코드', '동명칭', '대지구분코드', 'Cent_X',
       'Cent_Y'],
      dtype='object')

### (2) 건축물대장 기본개요

In [8]:
def make_sido_code(row):
    return str(row["시군구코드"])[:2]

In [10]:
%%time

# 건축물대장 기본개요
columns = ["관리건축물대장PK","관리상위건축물대장PK","대장구분코드","대장구분코드명","대장종류코드","대장종류코드명", \
            "대지위치","도로명대지위치","건물명","시군구코드","법정동코드","대지구분코드","번","지","특수지명","블록","로트","외필지수", \
            "새주소도로코드","새주소법정동코드","새주소지상지하코드","새주소본번","새주소부번","지역코드","지구코드","구역코드", \
            "지역코드명","지구코드명","구역코드명","생성일자"]

basis = dd.read_csv("data/rawdata/building/기본개요/mart_djy_01.txt", sep = "\|", engine='python', dtype=str, \
                    on_bad_lines='skip', keep_default_na=False, encoding = "cp949", header=None, names=columns)
print(f"건축물대장 기본개요의 분할된 개수는 {basis.npartitions}개입니다.")

busan_basis = basis[["관리건축물대장PK", "관리상위건축물대장PK", "시군구코드"]]

busan_basis["시도코드"] = busan_basis.apply(make_sido_code, axis=1, meta=object)
busan_basis = busan_basis[busan_basis["시도코드"] == "26"].drop(["시군구코드"], axis=1)

busan_basis = busan_basis.compute()

busan_basis.to_csv("data/refined-data/busan-표제부.csv", encoding="utf-8", index=False)

건축물대장 기본개요의 분할된 개수는 152개입니다.


/opt/homebrew/lib/python3.9/site-packages/dask/dataframe/core.py:6628: FutureWarning: Meta is not valid, `map_partitions` and `map_overlap` expects output to be a pandas object. Try passing a pandas object as meta or a dict or tuple representing the (name, dtype) of the columns. In the future the meta you passed will not work.
  warnings.warn(


CPU times: user 5min 42s, sys: 9.57 s, total: 5min 51s
Wall time: 5min 48s


### (3) 건축물대장 주택가격

In [1]:
%%time

# 건축물대장 주택가격

# names=columns
# columns = ["관리건축물대장PK","대장구분코드","대장구분코드명","대장종류코드","대장종류코드명","대지위치","도로명대지위치", \
#             "건물명","시군구코드","법정동코드","대지구분코드","번","지","특수지명","블록","로트","외필지수","새주소도로코드", \
#             "새주소지상지하코드","새주소본번","새주소부번","기준일자","주택가격","생성일자"]

price = dd.read_csv("data/rawdata/building/주택가격/mart_djy_08.txt", sep="\|", dtype=str, encoding='cp949',
                        on_bad_lines="skip", header=None, engine='python')
print(price.npartitions)

# the number of partitions
# print(f"526MB로 분할된 파일 개수는 {price.npartitions}개입니다.")

# save partitions
# price.to_csv("data/rawdata/building/주택가격/csvs", encoding="utf-8")

price_df = price[[0,8,23,24]]
price_df.columns = ["관리건축물대장PK","시군구코드","주택가격","생성일자"]
price_df["시도코드"] = price_df.apply(make_sido_code, axis=1, meta=object)
busan_price_pdf = price_df.loc[price_df["시도코드"] == "26"].compute()

# row 개수
# busan_price_pdf.map_partitions(len).compute()

busan_price_pdf.to_csv("data/refined-data/busan-price.csv", encoding="utf-8", index=False)

416
Wall time: 704 ms


### (3) 건축물대장 표제부

In [11]:
%%time

building = dd.read_csv("data/rawdata/building/기본개요/mart_djy_01.txt", sep = "\|", engine='python', dtype=str, \
                    on_bad_lines='skip', keep_default_na=False, encoding = "cp949", header=None)
print(f"건축물대장 기본개요의 분할된 개수는 {building.npartitions}개입니다.")

building.head()

건축물대장 기본개요의 분할된 개수는 152개입니다.
CPU times: user 1.4 s, sys: 149 ms, total: 1.55 s
Wall time: 1.61 s


,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,26380-4195,,1,일반,2,일반건축물,부산광역시 사하구 괴정동 193-103번지,부산광역시 사하구 대티로63번나길 4-1,,26380,...,0,4,1,1020,,,일반주거지역,,,20100619
1,26380-4200,,1,일반,2,일반건축물,부산광역시 사하구 괴정동 193-108번지,부산광역시 사하구 대티로63번길 30-5,,26380,...,0,30,5,1020,,,일반주거지역,,,20100619
2,26380-4260,26380-168,1,일반,2,일반건축물,부산광역시 사하구 괴정동 193-211번지,,,26380,...,0,,,1020,,,일반주거지역,,,20100619
3,26380-4292,26380-1169,1,일반,2,일반건축물,부산광역시 사하구 괴정동 193-295번지,,,26380,...,0,,,,,,,,,20100619
4,26380-4293,26380-1169,1,일반,2,일반건축물,부산광역시 사하구 괴정동 193-295번지,,,26380,...,0,,,,,,,,,20100619
